In [ ]:
import json
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
import _extractWebPage as ewp


def get_page_selenium(driver, url, isGetSubLink=True, delay=14):
    # 設置瀏覽器驅動
    driver.get(url)
    # 等待具有特定class属性的div标签出现并点击
    wait = WebDriverWait(driver, delay)
    all_data = []  # 存儲所有頁面的數據
    page_number = 1

    # 設置視窗大小，避免響應式設計影響元素位置
    # driver.set_window_size(1920, 1080)

    while True:
        try:
            print(f"正在處理第 {page_number} 頁")
            time.sleep(2)
            
            # 獲取當前頁面內容
            page_source = driver.page_source
            
            try:
                # 處理頁面內容
                pg = ewp.extractor(page_source, unwanted_tags=unwanted_tags, unwanted_elements=unwanted_elements, unwanted_texts= unwanted_text)
                

                if isGetSubLink == True:
                    # print(soup2)
                    base_url = 'https://www.esunbank.com/'
                    except_url = [] #visited_urls
                    all_links = ewp.find_all_links(pg.soup2, base_url, except_url)  # 這邊原來是 soup 全部的內容

                    # 將數據添加到列表中
                    page_data = {
                        'page_number': page_number,
                        'url': url,
                        'title': pg.title,
                        'markdown': pg.markdown_text,
                        'all_links': all_links,
                    }
                    all_data.append(page_data)
                    
                    # print(i_req, depth, url)
                    if len(all_links) > 0:
                        print('all-links', all_links)

                    # 過濾掉已經在 visited_urls 集合中的 URL
                    # all_links = [link for link in all_links if link not in visited_urls]
                    # for link in all_links:
                    #     i_req += 1
                    #     result = get_page_selenium(i_req, url, link, depth, max_depth, result, visited_urls)
                    
                     # # 這裡可以添加數據保存的邏輯
                    # 例如保存到文件：
                    with open(log_url, 'w', encoding='utf-8') as f:
                        json.dump(all_data, f, ensure_ascii=False, indent=2)

                print(f"成功提取第 {page_number} 頁數據")
            except Exception as e:
                print(f"處理第 {page_number} 頁內容時發生錯誤: {str(e)}")
            


            # 尋找並點擊下一頁按鈕
            try:
                next_button = wait.until(
                    EC.element_to_be_clickable((By.CSS_SELECTOR, "li.page-item.next"))
                    # EC.element_to_be_clickable((By.CSS_SELECTOR, "li a.group"))
                )
                
                # 檢查按鈕是否真的可見和可點擊
                if not next_button.is_displayed():
                    print("下一頁按鈕不可見，結束翻頁")
                    break

                # 確保按鈕在可視範圍內
                driver.execute_script("window.scrollTo(0, 0);") 
                time.sleep(1)
                #把next_button放在畫面的中間
                driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", next_button)
                time.sleep(1)
            
                # 使用 JavaScript 點擊
                driver.execute_script("arguments[0].click();", next_button)
                page_number += 1
                
                # 可以添加一個最大頁數限制
                if page_number > 7:  # 設置合適的最大頁數
                    print("達到最大頁數限制")
                    break
                    
            except TimeoutException:
                print("找不到下一頁按鈕，可能已經是最後一頁")
                break
            except Exception as e:
                print(f"點擊下一頁時發生錯誤: {str(e)}")
                break
                
        except Exception as e:
            print(f"處理頁面時發生未預期的錯誤: {str(e)}")
            break
    
    print(f"總共處理了 {page_number} 頁")
    return all_data




unwanted_tags = ['script', 'link', 'style', 'meta', 'noscript', 'iframe', 'head', 'header', 'footer', 'select', 'label', 'legend', 'form', 'img', 'input', 'button', 'table', 'figcaption']
unwanted_elements = {
    'div': ['hide-component', 'l-breadCrumb', 'bread', 'cookie-concent', 'gotop', 'menuBtn', 'darkLoad', 'market-widget', 'scroll-box', 'hotnews', 'important-info', 'nav-main__sub-menu', 'content__header', 'livenews__switch', 'article-function', 'fbmsg-box', 'box__promo', 'relatednews', 'ad-recommend', 'ad-box', 'theme-switch', 'l-bottomInfo__customerservice','l-warning'],
    'section': ['l-rating','l-cardService','l-bottomInfo'],
    'nav': ['paginationNav'],
    'p': ['hint', 'box-title'],
    'a': ['goMain', 'header__main-logo-square', 'nav-main__collapse-btn'],
    'ul': ['bread-crumb'],
    'li': ['nav-main__menu-item']
}
wanted_elements = {
    'ul': ['paginationList'],
}
unwanted_text = {
    'span': ['讀取中'],
    'a': [':::','進階篩選'],
    'h1': ['# 優惠總覽'],
    'p': ['消費生活首選，就愛刷玉山卡！']
}
# start_url = "https://www.esunbank.com/zh-tw/personal/credit-card/discount/shops/all"
# start_url = "https://www.esunbank.com/zh-tw/about/faq"
start_url = "https://www.esunbank.com/zh-tw/about/faq/faqlist?tag=credit-card"
log_url = "Result/玉山銀行faq_CreditCard/output.json"

def main():
    try:
        driver = webdriver.Chrome(options=ewp.init_chrome_option())
        all_pages_data = get_page_selenium(driver, start_url)
        # 處理收集到的數據
        print(f"總共收集到 {len(all_pages_data)} 頁的數據")
        
        # # 這裡可以添加數據保存的邏輯
        # # 例如保存到文件：
        # with open(log_url, 'w', encoding='utf-8') as f:
        #     json.dump(all_pages_data, f, ensure_ascii=False, indent=2)
        
    except Exception as e:
        print(f"主程序執行錯誤: {str(e)}")
    finally:
        # 如果需要關閉瀏覽器，取消下面的註釋
        # driver.quit()
        pass

if __name__ == "__main__":
    main()



In [ ]:
import json

# 讀取 JSON 資料
work_folder = "玉山銀行faq"
log_url = f"Result/{work_folder}/output.json"
out_url = f"Result/{work_folder}/output.all_links.json" 
with open(log_url, 'r', encoding='utf-8') as f:
    data = json.load(f)

# 提取 URL
urls = [item['all_links'] for item in data]

# 將 URL 轉換為集合，以去除重複項
unique_urls = set()
for sublist in urls:
    unique_urls.update(sublist)

# 對 URL 進行排序
sorted_urls = sorted(unique_urls)

# 將 URL 寫入文字檔
with open(out_url, 'w', encoding='utf-8') as f:
    for url in sorted_urls:
        f.write(url + '\n')



In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
import _extractWebPage as ewp

work_folder = "玉山銀行faq"
all_links_url = f"Result/{work_folder}/output.all_links.json" 

with open(all_links_url, 'r', encoding='utf-8') as f:
    urls = f.read().split('\n')

page_number = 0
driver = webdriver.Chrome(options=ewp.init_chrome_option())

unwanted_tags = ['script', 'link', 'style', 'meta', 'noscript', 'iframe', 'head', 'header', 'footer', 'select', 'label', 'legend', 'form', 'img', 'input', 'button', 'table', 'figcaption']
unwanted_elements = {
    'div': ['hide-component', 'l-breadCrumb', 'bread', 'cookie-concent', 'gotop', 'menuBtn', 'darkLoad', 'market-widget', 'scroll-box', 'hotnews', 'important-info', 'nav-main__sub-menu', 'content__header', 'livenews__switch', 'article-function', 'fbmsg-box', 'box__promo', 'relatednews', 'ad-recommend', 'ad-box', 'theme-switch', 'l-bottomInfo__customerservice','l-warning'],
    'section': ['l-rating','l-cardService','l-bottomInfo'],
    'nav': ['paginationNav'],
    'p': ['hint', 'box-title'],
    'a': ['goMain', 'header__main-logo-square', 'nav-main__collapse-btn'],
    'ul': ['bread-crumb'],
    'li': ['nav-main__menu-item']
}
unwanted_text = {
    'span': ['讀取中'],
    'a': [':::','進階篩選'],
    'h1': ['# 優惠總覽'],
    'p': ['消費生活首選，就愛刷玉山卡！']
}

for url in urls:
    driver.get(url)
    # wait = WebDriverWait(driver, 3)
    page_number += 1
    page_source = driver.page_source
    try:
        pg = ewp.extractor(page_source, unwanted_tags=unwanted_tags, unwanted_elements=unwanted_elements, unwanted_texts= unwanted_text)
        base_url = 'https://www.esunbank.com/'
        except_url = [] #visited_urls
        # all_links = ewp.find_all_links(pg.soup2, base_url, except_url)  # 這邊原來是 soup 全部的內容
        format_page_number = f"{page_number:04d}"
        
        # # 將數據添加到列表中
        # page_data = {
        #     'page_number': format_page_number,
        #     'url': url,
        #     'title': pg.title,
        #     'text': pg.extract_text,
        #     'markdown': pg.markdown_text,
        # }
        # print(f"抓取{format_page_number} {url} 中...")

        out_url = f"Result/{work_folder}/MD/{format_page_number}.{pg.title}.md"
        with open(out_url, 'w', encoding='utf-8') as f:
            f.write(f"參考網址：{url}\n\n")
            f.write(pg.markdown_text)
        
        out_url2 = f"Result/{work_folder}/TXT/{format_page_number}.{pg.title}.txt"
        with open(out_url2, 'w', encoding='utf-8') as f:
            f.write(f"參考網址：{url}\n\n")
            f.write(pg.extract_text)
            

        print(f"成功提取第 {page_number} 頁數據")
    except Exception as e:
        print(f"處理第 {page_number} {url} 頁內容時發生錯誤: {str(e)}")


SyntaxError: f-string: single '}' is not allowed (3246175747.py, line 56)